In [7]:
import torch
import scipy.optimize as sp
from functools import reduce

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# This function was copied from torch.optim.LBFGS
def _cubic_interpolate(x1, f1, g1, x2, f2, g2, bounds=None):
    # ported from https://github.com/torch/optim/blob/master/polyinterp.lua
    # Compute bounds of interpolation area
    if bounds is not None:
        xmin_bound, xmax_bound = bounds
    else:
        xmin_bound, xmax_bound = (x1, x2) if x1 <= x2 else (x2, x1)

    # Code for most common case: cubic interpolation of 2 points
    #   w/ function and derivative values for both
    # Solution in this case (where x2 is the farthest point):
    #   d1 = g1 + g2 - 3*(f1-f2)/(x1-x2);
    #   d2 = sqrt(d1^2 - g1*g2);
    #   min_pos = x2 - (x2 - x1)*((g2 + d2 - d1)/(g2 - g1 + 2*d2));
    #   t_new = min(max(min_pos,xmin_bound),xmax_bound);
    d1 = g1 + g2 - 3 * (f1 - f2) / (x1 - x2)
    d2_square = d1**2 - g1 * g2
    if d2_square >= 0:
        d2 = d2_square.sqrt()
        if x1 <= x2:
            min_pos = x2 - (x2 - x1) * ((g2 + d2 - d1) / (g2 - g1 + 2 * d2))
        else:
            min_pos = x1 - (x1 - x2) * ((g1 + d2 - d1) / (g1 - g2 + 2 * d2))
        return min(max(min_pos, xmin_bound), xmax_bound)
    else:
        return (xmin_bound + xmax_bound) / 2.

# This function was copied from torch.optim.LBFGS
# This function use the strong wolfe conditions to get optimal step length
def _strong_wolfe(obj_func,
                  x,
                  t,
                  d,
                  f,
                  g,
                  gtd,
                  c1=1e-4,
                  c2=0.9,
                  tolerance_change=1e-9,
                  max_ls=25):
    # ported from https://github.com/torch/optim/blob/master/lswolfe.lua
    d_norm = d.abs().max()
    g = g.clone(memory_format=torch.contiguous_format)
    # evaluate objective and gradient using initial step
    f_new, g_new = obj_func(x, t, d)
    ls_func_evals = 1
    gtd_new = g_new.dot(d)

    # bracket an interval containing a point satisfying the Wolfe criteria
    t_prev, f_prev, g_prev, gtd_prev = 0, f, g, gtd
    done = False
    ls_iter = 0
    while ls_iter < max_ls:
        # check conditions
        if f_new > (f + c1 * t * gtd) or (ls_iter > 1 and f_new >= f_prev):
            bracket = [t_prev, t]
            bracket_f = [f_prev, f_new]
            bracket_g = [g_prev, g_new.clone(memory_format=torch.contiguous_format)]
            bracket_gtd = [gtd_prev, gtd_new]
            break

        if abs(gtd_new) <= -c2 * gtd:
            bracket = [t]
            bracket_f = [f_new]
            bracket_g = [g_new]
            done = True
            break

        if gtd_new >= 0:
            bracket = [t_prev, t]
            bracket_f = [f_prev, f_new]
            bracket_g = [g_prev, g_new.clone(memory_format=torch.contiguous_format)]
            bracket_gtd = [gtd_prev, gtd_new]
            break

        # interpolate
        min_step = t + 0.01 * (t - t_prev)
        max_step = t * 10
        tmp = t
        t = _cubic_interpolate(
            t_prev,
            f_prev,
            gtd_prev,
            t,
            f_new,
            gtd_new,
            bounds=(min_step, max_step))

        # next step
        t_prev = tmp
        f_prev = f_new
        g_prev = g_new.clone(memory_format=torch.contiguous_format)
        gtd_prev = gtd_new
        f_new, g_new = obj_func(x, t, d)
        ls_func_evals += 1
        gtd_new = g_new.dot(d)
        ls_iter += 1

    # reached max number of iterations?
    if ls_iter == max_ls:
        bracket = [0, t]
        bracket_f = [f, f_new]
        bracket_g = [g, g_new]

    # zoom phase: we now have a point satisfying the criteria, or
    # a bracket around it. We refine the bracket until we find the
    # exact point satisfying the criteria
    insuf_progress = False
    # find high and low points in bracket
    low_pos, high_pos = (0, 1) if bracket_f[0] <= bracket_f[-1] else (1, 0)
    while not done and ls_iter < max_ls:
        # line-search bracket is so small
        if abs(bracket[1] - bracket[0]) * d_norm < tolerance_change:
            break

        # compute new trial value
        t = _cubic_interpolate(bracket[0], bracket_f[0], bracket_gtd[0],
                               bracket[1], bracket_f[1], bracket_gtd[1])

        # test that we are making sufficient progress:
        # in case `t` is so close to boundary, we mark that we are making
        # insufficient progress, and if
        #   + we have made insufficient progress in the last step, or
        #   + `t` is at one of the boundary,
        # we will move `t` to a position which is `0.1 * len(bracket)`
        # away from the nearest boundary point.
        eps = 0.1 * (max(bracket) - min(bracket))
        if min(max(bracket) - t, t - min(bracket)) < eps:
            # interpolation close to boundary
            if insuf_progress or t >= max(bracket) or t <= min(bracket):
                # evaluate at 0.1 away from boundary
                if abs(t - max(bracket)) < abs(t - min(bracket)):
                    t = max(bracket) - eps
                else:
                    t = min(bracket) + eps
                insuf_progress = False
            else:
                insuf_progress = True
        else:
            insuf_progress = False

        # Evaluate new point
        f_new, g_new = obj_func(x, t, d)
        ls_func_evals += 1
        gtd_new = g_new.dot(d)
        ls_iter += 1

        if f_new > (f + c1 * t * gtd) or f_new >= bracket_f[low_pos]:
            # Armijo condition not satisfied or not lower than lowest point
            bracket[high_pos] = t
            bracket_f[high_pos] = f_new
            bracket_g[high_pos] = g_new.clone(memory_format=torch.contiguous_format)
            bracket_gtd[high_pos] = gtd_new
            low_pos, high_pos = (0, 1) if bracket_f[0] <= bracket_f[1] else (1, 0)
        else:
            if abs(gtd_new) <= -c2 * gtd:
                # Wolfe conditions satisfied
                done = True
            elif gtd_new * (bracket[high_pos] - bracket[low_pos]) >= 0:
                # old high becomes new low
                bracket[high_pos] = bracket[low_pos]
                bracket_f[high_pos] = bracket_f[low_pos]
                bracket_g[high_pos] = bracket_g[low_pos]
                bracket_gtd[high_pos] = bracket_gtd[low_pos]

            # new point becomes new low
            bracket[low_pos] = t
            bracket_f[low_pos] = f_new
            bracket_g[low_pos] = g_new.clone(memory_format=torch.contiguous_format)
            bracket_gtd[low_pos] = gtd_new

    # return stuff
    t = bracket[low_pos]
    f_new = bracket_f[low_pos]
    g_new = bracket_g[low_pos]
    return f_new, g_new, t, ls_func_evals


class LSR1(torch.optim.Optimizer):
    """
    .. Class of the the limited memory symmetric rank-1 update. 
        The first six functions are from torch.optim.LBFGS. 
        The step function has some parts which is from torch.optim.LBFGS.

    .. warning::
        This optimizer doesn't support per-parameter options and parameter
        groups (there can be only one).

    .. note::
        This is a very memory intensive optimizer. If it doesn't fit in memory
        try reducing the history size, or use a different algorithm.

    Args:
        lr (float): learning rate (default: 0.2)
        max_iter (int): maximal number of iterations per optimization step
            (default: 20)
        max_eval (int): maximal number of function evaluations per optimization
            step (default: max_iter * 1.25).
        tolerance_grad (float): termination tolerance on first order optimality
            (default: 1e-5).
        tolerance_change (float): termination tolerance on function
            value/parameter changes (default: 1e-9).
        tr_rho (float): initial radius of trust region (default = 0.7).
        gamma (float): scalar of the initial hess matrix (default = 1).
        c_1,c_2,c_3 (float): parameters for update radius (default: c_1=2. c_2=0, c_3=0.5)
        history_size (int): update history size (default: 3).
        line_search_fn (str): either 'strong_wolfe' or None (default: None).
        trust_method (str): either 'cauchy' or 'CG' (default: cauchy)
    """

    def __init__(self,
                 params,
                 lr=0.2,
                 max_iter=20,
                 max_eval=None,
                 tolerance_grad=1e-15,
                 tolerance_change=1e-9,
                 tr_rho = 0.8,
                 history_size=7,
                 gamma=1,
                 c_1 = 2,
                 c_2 = 0,
                 c_3 = 0.5,
                 trust_CG_tol=1e-2,
                 mu_momentum = 0.5,
                 nu_momentum = 0.5,
                 alpha_S = 0.5,
                 trust_CG_iter=None,
                 line_search_fn="strong_wolfe",
                 trust_method=None):
        if max_eval is None:
            max_eval = max_iter * 5 // 4
        defaults = dict(
            lr=lr,
            max_iter=max_iter,
            max_eval=max_eval,
            tolerance_grad=tolerance_grad,
            tolerance_change=tolerance_change,
            tr_rho = tr_rho,
            history_size=history_size,
            gamma=gamma,
            c_1 = c_1,
            c_2 = c_2,
            c_3 = c_3,
            trust_CG_tol=trust_CG_tol,
            mu_momentum = mu_momentum,
            nu_momentum = nu_momentum,
            alpha_S = alpha_S,
            trust_CG_iter=trust_CG_iter,
            line_search_fn=line_search_fn,
            trust_method=trust_method)
        super(LSR1, self).__init__(params, defaults)

        # From torch.optim.LBFGS 
        # it checks if is one dictionary with params
        if len(self.param_groups) != 1:
            raise ValueError("LSR1 doesn't support per-parameter options "
                             "(parameter groups)")

        # From torch.optim.LBFGS
        # unpack the paramaters
        self._params = self.param_groups[0]['params']
        self._numel_cache = None
        self.history_size = self.param_groups[0]['history_size']

    #From torch.optim.LBFGS
    # im not sure what happens
    # its needed for the flat_grad
    def _numel(self):
        if self._numel_cache is None:
            self._numel_cache = reduce(lambda total, p: total + p.numel(), self._params, 0)
        return self._numel_cache

    #From torch.optim.LBFGS
    # flat the gradient
    def _gather_flat_grad(self):
        views = []
        for p in self._params:
            if p.grad is None:
                view = p.new(p.numel()).zero_()
            elif p.grad.is_sparse:
                view = p.grad.to_dense().view(-1)
            else:
                view = p.grad.view(-1)
            views.append(view)
        return torch.cat(views, 0)

    #From torch.optim.LBFGS
    # do the update 
    def _add_grad(self, step_size, update):
        offset = 0
        for p in self._params:
            numel = p.numel()
            # view as to avoid deprecated pointwise semantics
            p.add_(update[offset:offset + numel].view_as(p), alpha=step_size)
            offset += numel
        assert offset == self._numel()

    #From torch.optim.LBFGS
    def _clone_param(self):
        return [p.clone(memory_format=torch.contiguous_format) for p in self._params]

    #From torch.optim.LBFGS
    def _set_param(self, params_data):
        for p, pdata in zip(self._params, params_data):
            p.copy_(pdata)

    #From torch.optim.LBFGS
    def _directional_evaluate(self, closure, x, t, d):
        self._add_grad(t, d)
        loss = float(closure())
        flat_grad = self._gather_flat_grad()
        self._set_param(x)
        return loss, flat_grad

    def solve_trust_sub(self,flat_grad, hess_1, hess_2, tr_rho, trust_method, trust_CG_tol, trust_CG_iter):
      """
      .. The function solve a trust region subproblem.

      Args: 
          flat_grad (torch.Tensor): gradient vector
          hess (torch.Tensor): hesse matrix
          tr_tho (float): trust_radius
          trust_method (string): None, cauchy or CG. If None then cauchy
          trust_CG_tol: toleranz for reiduum for CG Steighaug
          trust:CG_iter: number of max iterations for CG Steighaug
      """
      # cauchy_condition : g^T * Hessematrix * g
      if trust_method ==None or trust_method=="cauchy":
        gH = torch.matmul(flat_grad, hess_1)
        Hg = torch.matmul(hess_2, flat_grad)
        cauchy_cond = torch.matmul(gH, Hg)
        if cauchy_cond <=0:
            tau = 1
        else:
            tau = min(torch.linalg.norm(flat_grad)**3/(cauchy_cond*tr_rho),1)
        return -tau*tr_rho/torch.linalg.norm(flat_grad)*flat_grad
      #performs CG Steighaug method for trust problem
      if trust_method == "CG":
        func = lambda a: torch.matmul(flat_grad, a)+torch.matmul(torch.matmul(a, hess_1),torch.matmul(hess_2, a))
        z = torch.ones(flat_grad.shape[0]).to(device)
        rr = flat_grad
        dd = -rr
        tol = trust_CG_tol
        if torch.linalg.norm(rr) < tol :
          return z
        for _ in range(trust_CG_iter):
          dH = torch.matmul(dd, hess_1)
          Hd = torch.matmul(hess_2, dd)
          dHd = torch.matmul(dH, Hd)
          dz = torch.matmul(dd,z)
          ddd = torch.matmul(dd,dd)
          zz = torch.matmul(z,z)
          root = abs(-(zz-tr_rho**2)/ddd + (dz/ddd)**2)
          no_root = dz/ddd
          tau_1 = -no_root + torch.sqrt(root)
          tau_2 = -no_root - torch.sqrt(root)
          tau = 1
          if dHd <= 0:
            if func(z+tau_1*dd)> func(z+tau_2*dd) :
              tau = tau_2
            else:
              tau = tau_1
            return z+tau*dd
          rrr = torch.matmul(rr,rr)
          alpha2 = rrr/dHd
          z_ersatz = z
          z = z+alpha2*dd
          if torch.linalg.norm(z) >= tr_rho:
            if tau_1 >= 0:
              tau = tau_1
            else: 
              tau = tau_2
            return z_ersatz+tau*dd
          rr = rr+ alpha2*torch.matmul(hess_1, torch.matmul(hess_2, dd))
          if torch.linalg.norm(rr) < tol:
            return z
          beta = torch.matmul(rr,rr)/rrr
          dd = -rr+beta*dd

    # calculate hess with limited memory method
    def calculate_hess(self, S, Y, gamma):
      """
      .. Calculate the hess matrix with the limited memory method.

      Args:
          S (torch.Tensor): saved s as a matrix, S.shape = (n, history_size)
          Y (torch.Tensor): saved y as a matrix, Y.shape = (n, history_size)
          gamma (float): skalar of initial hesse matrix
      """
      dim_hess = S.shape[0]

      # B_{k} = B_0 + phi * M^{-1} * phi
      phi = Y - gamma*S

      #calculate M = D+L+L^T-S*B_0*S
      SY = torch.mm(torch.transpose(S, 0, 1), Y)
      SS = gamma*torch.mm(torch.transpose(S, 0, 1), S)
      L = torch.tril(SY, diagonal=-1)
      M = L + torch.transpose(L, 0,1) + SS
      mask_M = range(M.shape[0])
      M[mask_M, mask_M] = M[mask_M, mask_M] + torch.diag(SY)

      #delete unnecessary matrices
      del Y
      del S       
      del SY
      del L
      del SS

      #check M singular, if yes then go to the next step
      if torch.det(M)==0:
          return False, 1, 1

      #calculate the inverse of M
      M_inverse = torch.linalg.solve(M, torch.eye(M.shape[0]).to(device)) 
      del M

      #thin q-r factorisation of phi
      Q, R = torch.linalg.qr(phi, mode="reduced")
      del phi

      # eigenvalues and eigenvectors of RM^{-1}R^T
      RMR = torch.mm(torch.mm(R, M_inverse), torch.transpose(R,0,1))
      lamb, U = torch.linalg.eig(RMR)
                
      del R
      del M_inverse

      # create last orthogonal matrix QU
      P = torch.mm(Q, U.float())
      del Q
      del U

      #create hess matrix, but dont calculate hess
      # save only the matrices hess_1 and hess_2 which hess = hess_1 @ hess_2
      P_1 = gamma+lamb.float()*P
      ones_column = torch.ones(P.shape[0]).reshape(-1,1).to(device)
      ones_row = torch.ones(P.shape[0]).reshape(1,-1).to(device)
      hess_1 = torch.cat((P_1, gamma*ones_column), 1).to(device)
      hess_2 = torch.cat((torch.transpose(P,0,1), ones_row),0).to(device)
      hess_2[-1, range(lamb.shape[0])]
      del P
      return True, hess_1, hess_2

    def update_SY(self, s, y, old_s, old_y, cond_rest):
      """
      .. Update S and Y. Pop the first if history_size is reached.

      Args:
          s (torch.Tensor): actual gradient vector
          y (torch.Tensor): previous gradient vector
          old_s (list): list with tensors last s
          old_y (list): list with tensors last y
          hess (torch.Tensor): hess matrix
          cond_rest (float): one part of the condition to update S or Y

      """
      ys = y.dot(s)  # y*s
      if ys + cond_rest > 1e-10:
          # updating memory
          if len(old_s) == self.history_size:
              # shift history by one (limited-memory)
              old_s.pop(0)
              old_y.pop(0)

          # store new direction/step
          old_s.append(s)
          old_y.append(y)

    #bearbeiten!!!!!!!
    def update_radius(self, r, c_1, c_2, c_3, tr_rho):
      """
      Args:
          r (float): ratio of actual and predicted reduction
          c_1,c_2,c_2 (flaot): hyperparameters of the changing radius
          tr_rho (float): trust radius 
      """
      if r <= c_2:
          tr_rho = c_3*tr_rho
      else : 
          tr_rho = c_1*tr_rho

    @torch.no_grad()
    def step(self, closure):
        """Performs a single optimization step.

        Args:
            closure (callable): A closure that reevaluates the model
                and returns the loss.
        """
        assert len(self.param_groups) == 1

        #From torch.optim.LBFGS
        # Make sure the closure is always called with grad enabled
        closure = torch.enable_grad()(closure)

        #load hyperparameter and settings
        group = self.param_groups[0]
        lr = group['lr']
        max_iter = group['max_iter']
        max_eval = group['max_eval']
        gamma = group['gamma']
        tolerance_grad = group['tolerance_grad']
        tolerance_change = group['tolerance_change']
        line_search_fn = group['line_search_fn']
        trust_method = group['trust_method']
        tr_rho = group['tr_rho']
        mu_momentum = group['mu_momentum']
        nu_momentum = group['nu_momentum']
        alpha_S = group['alpha_S']
        c_1 = group['c_1']
        c_2 = group['c_2']
        c_3 = group['c_3']
        trust_CG_tol = group['trust_CG_tol']
        trust_CG_iter = group['trust_CG_iter']
        
        #From torch.optim.LBFGS
        # NOTE: LSR1 has only global state, but we register it as state for
        # the first param, because this helps with casting in load_state_dict
        state = self.state[self._params[0]]
        state.setdefault('func_evals', 0)
        state.setdefault('n_iter', 0)

        #From torch.optim.LBFGS
        # evaluate initial f(x) and df/dx
        orig_loss = closure()
        loss = float(orig_loss)
        current_evals = 1
        state['func_evals'] += 1

        #From torch.optim.LBFGS
        #df/dx
        flat_grad = self._gather_flat_grad()
        opt_cond = flat_grad.abs().max() <= tolerance_grad

        #From torch.optim.LBFGS
        # optimal condition
        if opt_cond:
            return orig_loss

        # tensors cached in state (for tracing)
        d = state.get('d')
        v = state.get('v')
        alpha = state.get('alpha')
        old_s = state.get('old_s')
        old_y = state.get('old_y')
        prev_flat_grad = state.get('prev_flat_grad')
        prev_loss = state.get('prev_loss')
        tr_radius = state.get('tr_rho')
        s = state.get('s')

        dim_hess = flat_grad.shape[0]

        #check s is defined
        if s == None:
          s = torch.zeros(dim_hess)

        #check initial radius
        if tr_radius != None:
          tr_rho = tr_radius

        #check iterations of CG
        
        if trust_CG_iter == None:
          trust_CG_iter = dim_hess

        n_iter = 0
        # optimize for a max of max_iter iterations
        while n_iter < max_iter:
            # keep track of iterations
            n_iter += 1
            state['n_iter'] += 1

            ############################################################
            ####       compute gradient descent direction           ####
            ############################################################
            if state['n_iter'] == 1:
                # the first direction is the normal gradient
                # initialize parameters of the first step
                d = flat_grad.neg()
                old_s = []
                old_y = []
                hess_1 = gamma*torch.eye(dim_hess).to(device)
                hess_2 = torch.eye(dim_hess).to(device)
                tr_rho = tr_rho
                v = torch.zeros(dim_hess).to(device)
            else:
                # stack the list to a tensor 
                S = torch.transpose(torch.stack(old_s), 0, 1)
                Y = torch.transpose(torch.stack(old_y), 0, 1)

                # the approximate hess matrix
                cond_M, hess_1, hess_2 = self.calculate_hess(S,Y, gamma)
                #check if matrix "M" is singular
                if cond_M == False:
                  continue

                #solve trust region subproblem
                d = self.solve_trust_sub(flat_grad, hess_1, hess_2, tr_rho, trust_method, trust_CG_tol, trust_CG_iter)
                # do some other options: momentum etc.
                v = mu_momentum*v - nu_momentum*alpha_S*flat_grad+(1-nu_momentum)*s
                v = min(1, tr_rho/torch.linalg.norm(v))*v
                d = (1-nu_momentum)*d + mu_momentum*v
                d = min(1, tr_rho/torch.linalg.norm(d))*d
                d = d.to(device)
            dH = torch.matmul(d, hess_1)
            Hd = torch.matmul(hess_2, d)
            dHd = torch.matmul(dH, Hd)

            #############################################################
            #######     set lr, s, previous loss and flat_grad  #########
            #############################################################
            #From torch.optim.LBFGS
            # directional derivative
            gtd = flat_grad.dot(d)  # g * d

            # directional derivative is below tolerance
            if gtd > 0:
               d = -d

            #set s/alpha and a part of update condition/alpha
            # then we can delete hess
            s = d
            sH = torch.matmul(s, hess_1)
            Hs = torch.matmul(hess_2, s)
            cond_rest = torch.matmul(sH, Hs)
            del hess_1
            del hess_2

            #From torch.optim.LBFGS
            # reset initial guess for step size
            if state['n_iter'] == 1:
                alpha = min(1., 1. / flat_grad.abs().sum()) * lr
            else:
                alpha = lr

            # update prev_loss
            prev_loss = loss
            #update prev_flat_grad
            if prev_flat_grad is None:
                prev_flat_grad = flat_grad.clone(memory_format=torch.contiguous_format)
            else:
                prev_flat_grad.copy_(flat_grad)

            #############################################################
            #######               gradient step                 #########
            #############################################################
            #From torch.optim.LBFGS
            # optional line search: user function
            ls_func_evals = 0
            if line_search_fn is not None:
                # perform line search, using user function
                if line_search_fn != "strong_wolfe":
                    raise RuntimeError("only 'strong_wolfe' is supported")
                else:
                    x_init = self._clone_param()

                    def obj_func(x, t, d):
                        return self._directional_evaluate(closure, x, t, d)

                    loss, flat_grad, alpha, ls_func_evals = _strong_wolfe(
                        obj_func, x_init, alpha, d, loss, flat_grad, gtd)
                self._add_grad(alpha, d)
                opt_cond = flat_grad.abs().max() <= tolerance_grad
            else:
                # no line search, simply move with fixed-step
                self._add_grad(alpha, d)
                if n_iter != max_iter:
                    with torch.enable_grad():
                        loss = float(closure())
                    flat_grad = self._gather_flat_grad()
                    opt_cond = flat_grad.abs().max() <= tolerance_grad
                    ls_func_evals = 1
            y = flat_grad.sub(prev_flat_grad)
            #now we know alpha so we can use it
            s = alpha*s
            cond_rest = alpha*cond_rest

            #############################################################
            #######               update S,Y                    #########
            #############################################################
            self.update_SY(s, y, old_s, old_y, cond_rest)

            #############################################################
            #######               calculate ratio               #########
            #############################################################  
            ared = loss - prev_loss
            pred = loss + torch.matmul(flat_grad, d) + dHd
            r = ared/pred

            #############################################################
            #######               update radius                 #########
            #############################################################
            if state['n_iter']!= 1:
                self.update_radius(r, c_1, c_2, c_3, tr_rho)

            #From torch.optim.LBFGS
            # update func eval
            current_evals += ls_func_evals
            state['func_evals'] += ls_func_evals

            ############################################################
            #####               check conditions                  ######
            ############################################################
            #From torch.optim.LBFGS
            if n_iter == max_iter:
                break

            #From torch.optim.LBFGS
            if current_evals >= max_eval:
                break

            # optimal condition, gradient is not zero
            if opt_cond:
                break

            #From torch.optim.LBFGS
            # lack of progress
            if d.mul(alpha).abs().max() <= tolerance_change:
                break
            #From torch.optim.LBFGS
            if abs(loss - prev_loss) < tolerance_change:
                break

        state['d'] = d
        state['alpha'] = alpha
        state['old_s'] = old_s
        state['old_y'] = old_y
        state['prev_flat_grad'] = flat_grad
        state['prev_loss'] = loss
        state['tr_rho'] = tr_rho
        state['v'] = v
        state['s'] = s


        return orig_loss